In [1]:
# -- Cell 1: Imports & Configuration --
import os
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Paths
TEXT_JSON_DIR = os.path.expanduser(r"C:\Users\offic\AGENT\data\text_json")
INDEX_DIR     = os.path.expanduser(r"C:\Users\offic\AGENT\retriever\faiss_index")

# Ensure index directory exists
os.makedirs(INDEX_DIR, exist_ok=True)

print("Reading JSON from:", TEXT_JSON_DIR)
print("Index directory:", INDEX_DIR)


Reading JSON from: C:\Users\offic\AGENT\data\text_json
Index directory: C:\Users\offic\AGENT\retriever\faiss_index


In [2]:
# 1. Load all full_texts
docs = []
for fn in os.listdir(TEXT_JSON_DIR):
    if not fn.endswith(".json"):
        continue
    path = os.path.join(TEXT_JSON_DIR, fn)
    data = json.load(open(path, encoding="utf-8"))
    docs.append(data["full_text"])

print(f"Loaded {len(docs)} documents.")

# 2. Split into overlapping chunks (~1000 chars, 200 overlap)
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

chunks = []
for doc in docs:
    chunks.extend(splitter.split_text(doc))

print(f"Created {len(chunks)} text chunks.")
print("Max chunk length:", max(len(c) for c in chunks))


Loaded 67 documents.
Created 5120 text chunks.
Max chunk length: 1000


In [6]:
import time
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 1. Initialize embedding model (CPU only here; use device=0 for GPU)
emb = HuggingFaceEmbeddings(
    model_name="all-mpnet-base-v2",
    encode_kwargs={"batch_size": 64},           # pass batch_size via encode_kwargs
)

# 2. Build the FAISS index
print("Building FAISS index (this may take a few minutes)...")
t0 = time.time()
vectorstore = FAISS.from_texts(chunks, emb)
vectorstore.save_local(INDEX_DIR)
print(f"✅ Index built & saved in {time.time() - t0:.1f}s")


Building FAISS index (this may take a few minutes)...
✅ Index built & saved in 2437.1s


In [8]:
from langchain.text_splitter import TokenTextSplitter
from transformers import AutoTokenizer

# Re‑split your raw docs into ~500‑token chunks:
tokenizer = AutoTokenizer.from_pretrained("all-mpnet-base-v2")
splitter = TokenTextSplitter(
    tokenizer=tokenizer,
    chunk_size=500,      # tokens
    chunk_overlap=50,    # tokens
)

chunks = []
for doc in docs:  # docs from Cell 2
    chunks.extend(splitter.split_text(doc))

print(f"Token splitter → {len(chunks)} chunks; max tokens in a chunk:",
      max(len(tokenizer.encode(c)) for c in chunks))


OSError: all-mpnet-base-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [4]:
#!/usr/bin/env python3
"""
retriever_embeddings.py

1) Load JSON‑extracted paper texts from data/text_json/
2) Split into ~500‑token chunks via TokenTextSplitter(encoding_name="cl100k_base")
3) Embed chunks with sentence-transformers/all-mpnet-base-v2 (CPU)
4) Build & save a FAISS index under retriever/faiss_index/
5) Reload the index and run a sample similarity search
"""

import os
import json
import time
from transformers import AutoTokenizer
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# ─── CONFIG ────────────────────────────────────────────────────────────────────

TEXT_JSON_DIR      = r"C:\Users\offic\AGENT\data\text_json"
INDEX_DIR          = r"C:\Users\offic\AGENT\retriever\faiss_index"
EMBED_MODEL        = "sentence-transformers/all-mpnet-base-v2"

TOKEN_CHUNK_SIZE    = 500
TOKEN_CHUNK_OVERLAP = 50

EMBED_BATCH_SIZE    = 64

# ─── PREPARE ───────────────────────────────────────────────────────────────────

os.makedirs(INDEX_DIR, exist_ok=True)
print(f"Reading JSON from: {TEXT_JSON_DIR}")
print(f"Index will be saved to: {INDEX_DIR}\n")

# ─── STEP 1: LOAD DOCUMENTS ────────────────────────────────────────────────────

docs = []
for fname in sorted(os.listdir(TEXT_JSON_DIR)):
    if fname.endswith(".json"):
        path = os.path.join(TEXT_JSON_DIR, fname)
        with open(path, encoding="utf-8") as f:
            data = json.load(f)
            docs.append(data.get("full_text", ""))
print(f"Loaded {len(docs)} documents.\n")

# ─── STEP 2: TOKEN‑BASED SPLITTING ──────────────────────────────────────────────

print(f"Splitting into ~{TOKEN_CHUNK_SIZE}-token chunks (+{TOKEN_CHUNK_OVERLAP} overlap)")
splitter = TokenTextSplitter(
    encoding_name="cl100k_base",
    chunk_size=TOKEN_CHUNK_SIZE,
    chunk_overlap=TOKEN_CHUNK_OVERLAP,
)

chunks = []
for d in docs:
    chunks.extend(splitter.split_text(d))
print(f"Created {len(chunks)} token chunks.")

# (Optional) inspect chunk sizes
tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL)
lengths = [len(tokenizer.encode(c)) for c in chunks]
print(f"Max tokens/chunk: {max(lengths)}, mean: {sum(lengths)/len(lengths):.1f}\n")

# ─── STEP 3: EMBEDDINGS & FAISS INDEX ──────────────────────────────────────────

print("Initializing embeddings model (CPU):", EMBED_MODEL)
emb = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"batch_size": EMBED_BATCH_SIZE},
)

print("Building FAISS index (this may take a few minutes)...")
t0 = time.time()
vectorstore = FAISS.from_texts(chunks, emb)
vectorstore.save_local(INDEX_DIR)
print(f"✅ Index built & saved in {time.time() - t0:.1f}s\n")

# ─── STEP 4: LOAD & TEST RETRIEVAL ─────────────────────────────────────────────

print("Reloading FAISS index with pickle deserialization enabled")
vs = FAISS.load_local(
    INDEX_DIR,
    emb,
    allow_dangerous_deserialization=True
)
print("Index loaded. Running sample query...")

query = "Which architectures used ResNet?"
results = vs.similarity_search(query, k=5)

print("\nTop 5 chunks for query:")
for i, doc in enumerate(results, 1):
    snippet = doc.page_content.replace("\n", " ")
    print(f"{i}. {snippet[:200]}…\n")


Reading JSON from: C:\Users\offic\AGENT\data\text_json
Index will be saved to: C:\Users\offic\AGENT\retriever\faiss_index

Loaded 67 documents.

Splitting into ~500-token chunks (+50 overlap)
Created 2454 token chunks.


Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


Max tokens/chunk: 635, mean: 463.3

Initializing embeddings model (CPU): sentence-transformers/all-mpnet-base-v2
Building FAISS index (this may take a few minutes)...
✅ Index built & saved in 1138.7s

Reloading FAISS index with pickle deserialization enabled
Index loaded. Running sample query...

Top 5 chunks for query:
1. 44 ± 0.77 60.60 ± 0.71 Prototypical Networks (Snell et al., 2017) conv (64)×4 49.42 ± 0.78 68.20 ± 0.66 MAML (Finn et al., 2017) conv (32)×4 48.70 ± 1.84 63.11 ± 0.92 R2D2 (Bertinetto et al., 2018) co…

2. V  Output Input Input Output Output + Path A Path B Figure 1: The architecture of ResNet-50. The convolution kernel size, output channel size and stride size (default is 1) are illustrated, simila…

3.  82.32±0.14 71.66±0.23 85.50±0.15 DMF [66] ResNet-12 12.4 M 67.76±0.46 82.71±0.31 71.89±0.52 85.96±0.35 InfoPatch [39] ResNet-12 12.4 M 67.67±0.45 82.44±0.31 - - BML [77] ResNet-12 12.4 M 67.04±0.63 8…

4. ×1, 1024  ×36 conv5 x 7×7  3×3, 512 3×3, 512  ×2  3×